# Distinguishing k-regular graphs

Multiple graph structures are not distinguishable by the standard message
passing Graph Neural Networks.
For instance, the Graph Neural Networks are not able to distinguish between
_k_-regular graphs of the same size, such as the pair shown in the following image. Those graphs are not isomorphic and are both $3$-regular, meaning all nodes have precisely three neighbors.

![3-Regular Graphs](https://raw.githubusercontent.com/LukasZahradnik/PyNeuraLogic/master/docs/_static/k_regular_graph.png)

In [1]:
from neuralogic.nn import get_evaluator
from neuralogic.core import Backend
from neuralogic.core import Relation, Template, Var, Term
from neuralogic.core.settings import Settings, Optimizer
from neuralogic.utils.data import Dataset

When we assign the same features to all nodes, the messages during the
update step of message passing GNNs will be identical, resulting in
the same features and eventually classifying both graphs as the same class.
Such misclassification can be problematic in multiple domains, e.g.,
chemistry, where two indistinguishable graphs represent two different molecules.

Via the PyNeuraLogic library, we are able to embed the pattern of both graphs
or their parts. There are many alternative approaches to distinguish between
those two graphs; our presented example utilizes encoding of triangles to capture
triangles of graph _b_, with additional rules aggregating the
general graph structure.


In [2]:
train_dataset = Dataset()
template = Template()

template.add_rules([
    # Captures triangle
    Relation.triangle(Var.X)[1,] <= (
        Relation.edge(Var.X, Var.Y), Relation.feature(Var.Y)[1,],
        Relation.edge(Var.Y, Var.Z), Relation.feature(Var.Z)[1,],
        Relation.edge(Var.Z, Var.X), Relation.feature(Var.X)[1,],
    ),

    # Captures general graph
    Relation.general(Var.X)[1,] <= (Relation.edge(Var.X, Var.Y), Relation.feature(Var.Y)[1,]),
    Relation.general(Var.X)[1,] <= Relation.feature(Var.Y)[1,],

    Relation.predict <= Relation.general(Var.X)[1,],
    Relation.predict <= Relation.triangle(Var.X)[1,],
])

train_dataset.add_example(
    [
        Relation.edge(1, 2), Relation.edge(2, 3), Relation.edge(3, 4), Relation.edge(4, 1),
        Relation.edge(2, 1), Relation.edge(3, 2), Relation.edge(4, 3), Relation.edge(1, 4),
        Relation.edge(1, 6), Relation.edge(3, 6), Relation.edge(4, 5), Relation.edge(2, 5),
        Relation.edge(6, 1), Relation.edge(6, 3), Relation.edge(5, 4), Relation.edge(5, 2),

        Relation.feature(1), Relation.feature(2), Relation.feature(3),
        Relation.feature(4), Relation.feature(5), Relation.feature(6),
    ],
)

train_dataset.add_example(
    [
        Relation.edge(1, 2), Relation.edge(2, 3), Relation.edge(3, 4), Relation.edge(4, 1),
        Relation.edge(2, 1), Relation.edge(3, 2), Relation.edge(4, 3), Relation.edge(1, 4),
        Relation.edge(1, 6), Relation.edge(4, 6), Relation.edge(3, 5), Relation.edge(2, 5),
        Relation.edge(6, 1), Relation.edge(6, 4), Relation.edge(5, 3), Relation.edge(5, 2),

        Relation.feature(1), Relation.feature(2), Relation.feature(3),
        Relation.feature(4), Relation.feature(5), Relation.feature(6),
    ],
)

train_dataset.add_queries([
    Relation.predict[1],
    Relation.predict[0],
])

In [3]:
settings = Settings(optimizer=Optimizer.SGD, epochs=200)
neuralogic_evaluator = get_evaluator(template, Backend.DYNET, settings)

for _ in neuralogic_evaluator.train(train_dataset):
    pass

graphs = ["a", "b"]

for graph_id, (label, predicted) in enumerate(neuralogic_evaluator.test(train_dataset)):
    print(f"Graph {graphs[graph_id]} is predicted to be class: {int(round(predicted))} | {predicted}")

[dynet] random seed: 2995899108
[dynet] allocating memory: 512MB
[dynet] memory allocation done.


Graph a is predicted to be class: 1 | 0.9270843863487244
Graph b is predicted to be class: 0 | 0.02791881561279297
